In [ ]:
""" Main Script to evaluate a trained model """
print 


In [ ]:
import torch
torch.__version__

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import math
import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 

In [ ]:
# LOAD THE PRETRAINED (DEFENDED) MODEL (NOTE: THIS HAS TO BE RESNET32)

USE_GPU = torch.cuda.is_available()
EXPERIMENT_NAME = 'linf_flow'


classifier_net, cifar_normer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, use_gpu=USE_GPU, 
                                                                    return_normalizer=True) 


defended_state_dict_file = checkpoints.params_to_filename(EXPERIMENT_NAME, 'resnet32')[-1]
print "Loading %s params into resnet32" % defended_state_dict_file 

classifier_net = checkpoints.load_state_dict_from_filename(defended_state_dict_file, classifier_net)

In [ ]:
# LOAD THE EVALUATION DATASET 
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=128, use_gpu=USE_GPU, 
                                         shuffle=False)


In [ ]:
reload(adveval)
# Build various attacks to test 
L_INF_BOUND = 8.0/255.0 
FLOW_LINF = 0.05
PGD_ITER = 50 


##############################################################################
#                                                                            #
#             FGSM ONLY                                                      #
#                                                                            #
##############################################################################

def build_delta_fgsm_eval():
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND, 
                                                            use_gpu=USE_GPU))
    attack_loss = plf.VanillaXentropy(classifier_net, cifar_normer)
    fgsm_attack = aar.FGSM(classifier_net, cifar_normer, delta_threat, attack_loss, use_gpu=USE_GPU)
    attack_kwargs ={'verbose': False}
    params = advtrain.AdversarialAttackParameters(fgsm_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': attack_kwargs})    
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result 


##############################################################################
#                                                                            #
#        PGD LINFINITY                                                       #
#                                                                            #
##############################################################################


    
def build_delta_pgd_eval():
    # UNSIGNED PGD ATTACK 
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND,
                                                            use_gpu=USE_GPU))
    attack_loss = plf.VanillaXentropy(classifier_net, cifar_normer)
    pgd_attack = aar.PGD(classifier_net, cifar_normer, delta_threat, attack_loss, use_gpu=USE_GPU)
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.01}
    pgd_kwargs = {'num_iterations': PGD_ITER, 
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}      
    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result     

##############################################################################
#                                                                            #
#      PGD FLOW                                                              #
#                                                                            #
##############################################################################

def build_stadv_pgd_eval():
    # STAdv style attack 
    flow_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                 ap.PerturbationParameters(lp_style='inf', 
                                                           lp_bound=FLOW_LINF,
                                                           xform_class=st.FullSpatial, 
                                                           use_gpu=USE_GPU,
                                                           use_stadv=True))
    adv_loss = lf.CWLossF6(classifier_net, cifar_normer)
    st_loss = lf.PerturbationNormLoss(lp=2)
    
    loss_fxn = lf.RegularizedLoss({'adv': adv_loss, 'st':st_loss}, 
                                  {'adv': 1.0, 'st': 0.05},
                                  negate=True)  
    
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.001}
    pgd_attack = aar.PGD(classifier_net, cifar_normer, flow_threat, loss_fxn, use_gpu=USE_GPU)
    pgd_kwargs = {'num_iterations': PGD_ITER,  
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}    


    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result  


##############################################################################
#                                                                            #
#     PGD + ROT + TRANS                                                      #
#                                                                            #
##############################################################################

def build_delta_rot_trans_pgd_eval():
    # L_inf + flow style attack 
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND,
                                                            use_gpu=USE_GPU))
    trans_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                  ap.PerturbationParameters(lp_style=1, 
                                                            lp_bound=0.05, 
                                                            xform_class=st.TranslationTransform,
                                                            use_gpu=USE_GPU))
    rotation_threat = ap.ThreatModel(ap.ParameterizedXformAdv,
                                     ap.PerturbationParameters(xform_class=st.RotationTransform, 
                                                              lp_style='inf', lp_bound=math.pi / 24.,
                                                              use_gpu=USE_GPU))
    
    sequence_threat = ap.ThreatModel(ap.SequentialPerturbation, 
                                 [delta_threat, trans_threat, rotation_threat])

    loss_fxn = plf.VanillaXentropy(classifier_net, cifar_normer)
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.001}
    pgd_attack = aar.PGD(classifier_net, cifar_normer, sequence_threat, loss_fxn, use_gpu=USE_GPU)
    pgd_kwargs = {'num_iterations': PGD_ITER, 
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}    

    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result 



##############################################################################
#                                                                            #
#    DELTA + FLOW                                                            #
#                                                                            #
##############################################################################

def build_delta_stadv_pgd_eval():
    # L_inf + flow style attack 
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND, 
                                                            use_gpu=USE_GPU))
    flow_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                 ap.PerturbationParameters(lp_style='inf', 
                                                           lp_bound=FLOW_LINF,
                                                           xform_class=st.FullSpatial, 
                                                           use_gpu=USE_GPU,
                                                           use_stadv=True))
    sequence_threat = ap.ThreatModel(ap.SequentialPerturbation, 
                                 [delta_threat, flow_threat],
                                ap.PerturbationParameters(norm_weights=[0.00, 1.00]))
    adv_loss = lf.CWLossF6(classifier_net, cifar_normer)
    st_loss = lf.PerturbationNormLoss(lp=2)
    
    loss_fxn = lf.RegularizedLoss({'adv': adv_loss, 'st':st_loss}, 
                                  {'adv': 1.0, 'st': 0.05},
                                  negate=True)
    
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.001}
    pgd_attack = aar.PGD(classifier_net, cifar_normer, sequence_threat, loss_fxn, use_gpu=USE_GPU)
    pgd_kwargs = {'num_iterations': PGD_ITER, 
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}    

    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result 



##############################################################################
#                                                                            #
#    DELTA + FLOW + ROT + TRANS                                              #
#                                                                            #
##############################################################################

def build_delta_stadv_rot_trans_pgd_eval():
    # L_inf + flow style attack 
    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND, 
                                                            use_gpu=USE_GPU))
    flow_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                 ap.PerturbationParameters(lp_style='inf', 
                                                           lp_bound=FLOW_LINF,
                                                           xform_class=st.FullSpatial, 
                                                           use_gpu=USE_GPU,
                                                           use_stadv=True))
    
    trans_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                                  ap.PerturbationParameters(lp_style=1, 
                                                            lp_bound=0.05, 
                                                            xform_class=st.TranslationTransform,
                                                            use_gpu=USE_GPU))
    rotation_threat = ap.ThreatModel(ap.ParameterizedXformAdv,
                                     ap.PerturbationParameters(xform_class=st.RotationTransform, 
                                                              lp_style='inf', lp_bound=math.pi / 24.,
                                                              use_gpu=USE_GPU))
    
    sequence_threat = ap.ThreatModel(ap.SequentialPerturbation, 
                                 [delta_threat, flow_threat, trans_threat, rotation_threat],
                                ap.PerturbationParameters(norm_weights=[0.00, 1.00, 1., 1.]))
    adv_loss = lf.CWLossF6(classifier_net, cifar_normer)
    st_loss = lf.PerturbationNormLoss(lp=2)
    
    loss_fxn = lf.RegularizedLoss({'adv': adv_loss, 'st':st_loss}, 
                                  {'adv': 1.0, 'st': 0.05},
                                  negate=True)
    
    optimizer = optim.Adam
    optimizer_kwargs = {'lr': 0.001}
    pgd_attack = aar.PGD(classifier_net, cifar_normer, sequence_threat, loss_fxn, use_gpu=USE_GPU)
    pgd_kwargs = {'num_iterations': PGD_ITER, 
                  'signed': False, 
                  'optimizer': optimizer, 
                  'optimizer_kwargs': optimizer_kwargs, 
                  'verbose': False}    

    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                       attack_specific_params={'attack_kwargs': pgd_kwargs})
    to_eval= {'top1': 'top1',
              'lpips': 'avg_successful_lpips'}
    
    eval_result = adveval.EvaluationResult(params, classifier_net, cifar_normer, 
                                           to_eval=to_eval, use_gpu=USE_GPU)
    return eval_result 



fgsm = build_delta_fgsm_eval()
pgd_linf = build_delta_pgd_eval()
stadv = build_stadv_pgd_eval()
delta_rot_trans = build_delta_rot_trans_pgd_eval()
delta_stadv = build_delta_stadv_pgd_eval() 
full = build_delta_stadv_rot_trans_pgd_eval()


In [ ]:
def verify_eval(eval_obj, num=4):
    examples, labels = next(iter(val_loader))
    examples = examples.cuda()
    labels = labels.cuda() 
    attack_out = eval_obj.attack_params.attack(examples, labels)
    img_utils.show_images([attack_out[0][:num], examples[:num], 
                           attack_out[0][:num] - examples[:num] + 0.5])

In [ ]:
# BUILD ATTACK ENSEMBLE 
ensemble = {
            'fgsm': fgsm, 
            'pgd_linf': pgd_linf,
            'stadv': stadv,
            'delta_rot_trans': delta_rot_trans, 
            'delta_stadv': delta_stadv,
            'full': full
           }
            
eval_obj = adveval.AdversarialEvaluation(classifier_net, cifar_normer, use_gpu=USE_GPU)
eval_output = eval_obj.evaluate_ensemble(val_loader, ensemble, verbose=True, num_minibatches=10)


In [ ]:
print EXPERIMENT_NAME
for k, v in eval_output.items():
    if k != 'ground':
        print k, v.results['top1'].avg, v.results['lpips']
    else:
        print k, v.results['top1'].avg

In [ ]:
#reload(st)
stadv = build_stadv_pgd_eval()
classifier_net.cuda()
verify_eval(full, num=4)
